In [1]:
import pathlib
import torch
import torch.nn as nn

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors, Regex
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace, Split
from tokenizers.trainers import BpeTrainer
from torch.nn import functional as F

In [2]:
title = "FolkAltRock"

In [3]:
train_test_text = 'trainTestData/FolkAltRock/trainTest.txt'
train_text = 'trainTestData/FolkAltRock/train.txt'
test_text = 'trainTestData/FolkAltRock/test.txt'

In [4]:
# BPE vocabulary
vocab_size = 10000
min_frequency=2

#hyperparameters
training_iterations = 5000
batch_size = 64 # independent sequences processed in parallel
block_size = 256 #max context length for predictions
step = 0
eval_interval = 500
learning_rate = 5e-4
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [5]:
torch.backends.mps.is_available()

True

In [6]:
with open(train_test_text, 'r', encoding='utf-8') as f:
    text = f.read()

with open(train_text, 'r', encoding='utf-8') as f:
    train_text = f.read()

with open(test_text, 'r', encoding='utf-8') as f:
    test_text = f.read()

In [7]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Split(Regex('[ \t]+'), behavior='removed')
trainer = BpeTrainer(vocab_size=vocab_size, min_frequency=min_frequency)
tokenizer.train(files=[train_test_text], trainer=trainer)

In [9]:
vocab_size = tokenizer.get_vocab_size()
encode = lambda s: tokenizer.encode(s).ids
decode = lambda l: tokenizer.decode(l)

In [10]:
y = text[:250]
print(y)
print("---")
print(len(y))
x = encode(y)
print(len(x))
print(x)

Yesterday, all my troubles seemed so far away
Now it looks as though they're here to stay
Oh, I believe in yesterday Suddenly, I'm not half the man I used to be
There's a shadow hanging over me.
Oh, yesterday came suddenly Why she had to go I don't k
---
250
51
[9426, 128, 136, 3890, 6183, 199, 1154, 655, 410, 112, 2745, 140, 1475, 1277, 211, 108, 2090, 388, 34, 746, 96, 2782, 9770, 8, 188, 277, 2127, 99, 246, 34, 869, 108, 648, 696, 55, 4228, 1966, 431, 1789, 388, 2782, 859, 9658, 1011, 255, 397, 108, 129, 34, 208, 65]


In [11]:
print(vocab_size)

10000


In [12]:
print(decode(x))

Yesterday, all my troubles seemed so far away
 Now it looks as though they're here to stay
 Oh, I believe in yesterday Suddenly , I'm not half the man I used to be
 There's a shadow hanging over me.
 Oh, yesterday came suddenly Why she had to go I don't k


In [13]:
train_data = torch.tensor(encode(train_text), dtype=torch.long)
val_data = torch.tensor(encode(test_text), dtype=torch.long)

In [14]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        #compute attn scores ('affinities')
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        
        #perform the weighted aggregation of the values
        v = self.value(x) # (B, T, C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [17]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of attention in parallel"""
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [18]:
class FeedForward(nn.Module):
    """ a simple linear leayer followed by a non-linearity"""
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

In [19]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation"""
    
    def __init__(self, nembd, n_head):
        # n_embd: embedding dimension, n_head: th number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [20]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) #number of embedding dimensions
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # postion of each char
        
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        
        self.lm_head = nn.Linear(n_embd, vocab_size) # language model head
    
    def forward(self, idx, targets = None):
        B, T = idx.shape
        
        #idx and targets are both (B,T) tensor of ints
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb= self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to probs
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled indext to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [21]:
def create_snapshots():
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'losses': losses,
            }, str(pathlib.Path().resolve()) + f'/{title}_snapshot_at_step{step}')
    tokenizer.save(f'{title}_tokenizer_at_step_{step}.json')

In [22]:
model = BigramLanguageModel()
m = model.to(device)

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for itr in range(training_iterations + 1):
    # every once in a while evaluate the loss on the train and val sets:
    if itr % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        create_snapshots()
    
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    step += 1

Step 0: train loss 9.3804, val loss 9.3783
Step 500: train loss 3.0269, val loss 6.8593
Step 1000: train loss 0.8594, val loss 8.3349
Step 1500: train loss 0.2885, val loss 9.5801
Step 2000: train loss 0.1814, val loss 10.3594
